In [2]:
import numpy as np
import cv2
import torch
import torchvision
from PIL import Image
import os
import copy
import torch
import torchvision
import torch.nn as nn
import scipy
import torchvision.transforms as transforms
from torchvision import datasets as ds
from torch.utils.data import DataLoader,Dataset,Subset,ConcatDataset,random_split
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import tqdm
import celeba_dataset

In [3]:
print(os.getcwd())

/home/zhixin/Project/FL_CelebA_10_27


In [4]:
# 把数据缩放到（-1，1）
class Oneone(torch.nn.Module):
    def __init__(self, inplace=False):
        super().__init__()
        self.inplace = inplace

    def forward(self, tensor):
        return tensor*2.0-1.0
        # return F.normalize(tensor, self.mean, self.std, self.inplace)

# transform = transforms.Compose是把一系列图片操作组合起来，比如减去像素均值等。
# DataLoader读入的数据类型是PIL.Image
# 这里对图片不做任何处理，仅仅是把PIL.Image转换为torch.FloatTensor，从而可以被pytorch计算
transform_train = transforms.Compose(
    [
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        Oneone(),
    ]
)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    Oneone(),
])

In [5]:
load_model = True
load_data_loader = False
learning_rate = 0.005
batch_size = 128
trigger_size = 8
trigger_pos = 0
inject_r = 0.1
untrust_prop = 0.95
ret = 175# ret是控制mask透明度的阈值（175）
target_label_1 = 9
target_label_2 = 0

In [6]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=2, init_weights=False):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(  #打包
            nn.Conv2d(3, 48, kernel_size=11, stride=4, padding=2),  # input[3, 224, 224]  output[48, 55, 55] 自动舍去小数点后
            nn.ReLU(inplace=True), #inplace 可以载入更大模型
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[48, 27, 27] kernel_num为原论文一半
            nn.Conv2d(48, 128, kernel_size=5, padding=2),           # output[128, 27, 27]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 13, 13]
            nn.Conv2d(128, 192, kernel_size=3, padding=1),          # output[192, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=3, padding=1),          # output[192, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 128, kernel_size=3, padding=1),          # output[128, 13, 13]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 6, 6]
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            #全链接
            nn.Linear(128 * 6 * 6, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(2048, 2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1) #展平   或者view()
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu') #何教授方法
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)  #正态分布赋值
                nn.init.constant_(m.bias, 0)

In [7]:
net = AlexNet(init_weights=True)
print(net)
# 定义损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()
#优化器 这里用Adam
optimizer = torch.optim.Adam(net.parameters(), lr=0.0002)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# 如果有gpu就使用gpu，否则使用cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda:1')
net = net.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 48, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(48, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(192, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=4608, out_features=2048, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, 

#  加载CelebA数据集

In [8]:
#加载原始数据集
celebA_train_dataset=celeba_dataset.CelebA(root='../', split='train',target_type='attr', transform=transform_train, target_transform=None, download=True)
celebA_val_dataset=celeba_dataset.CelebA(root='../', split='test',target_type='attr',transform=transform_train, target_transform=None, download=True)

#取部分数据集
celebA_train_dataset=random_split(celebA_train_dataset,
                                  lengths=[int(len(celebA_train_dataset)*0.25),len(celebA_train_dataset)-int(len(celebA_train_dataset)*0.25)])[0]

celebA_train_loader=DataLoader(dataset = celebA_train_dataset,
                              batch_size=batch_size,
                              shuffle=True)
celebA_val_loader=DataLoader(dataset = celebA_val_dataset,
                              batch_size=batch_size,
                              shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
print("标签参数如下")
for i,name in enumerate(celebA_val_dataset.attr_names):
    print(str(i)+" "+name)

print("训练集长度："+str(len(celebA_train_dataset)) )
print("验证集长度："+str(len(celebA_val_dataset)) )

标签参数如下
0 5_o_Clock_Shadow
1 Arched_Eyebrows
2 Attractive
3 Bags_Under_Eyes
4 Bald
5 Bangs
6 Big_Lips
7 Big_Nose
8 Black_Hair
9 Blond_Hair
10 Blurry
11 Brown_Hair
12 Bushy_Eyebrows
13 Chubby
14 Double_Chin
15 Eyeglasses
16 Goatee
17 Gray_Hair
18 Heavy_Makeup
19 High_Cheekbones
20 Male
21 Mouth_Slightly_Open
22 Mustache
23 Narrow_Eyes
24 No_Beard
25 Oval_Face
26 Pale_Skin
27 Pointy_Nose
28 Receding_Hairline
29 Rosy_Cheeks
30 Sideburns
31 Smiling
32 Straight_Hair
33 Wavy_Hair
34 Wearing_Earrings
35 Wearing_Hat
36 Wearing_Lipstick
37 Wearing_Necklace
38 Wearing_Necktie
39 Young
40 
训练集长度：40692
验证集长度：19962


In [10]:
#数据切分加工
def get_split_indices(dataset,target_index):
    '''
    按标签划分数据集，返回数据集的下标集合
    :param dataset:
    :param target_index: 按照这个下标的标签进行划分
    :return: 返回的list中，每个元素代表不同值的标签下标的集合
    '''
    group1=[]
    group2=[]
    for i,(data,target) in enumerate(dataset):
        if target[target_index]==0:
            group1.append(i)
        else:
            group2.append(i)
    return [group1,group2]
def shuffle_dataset(dataset,target_index):
    '''
    打乱数据集，把目标标签取反
    :param dataset:
    :param target_index: 标签下标
    :return:
    '''
    dataset=copy.deepcopy(dataset)
    for i,(data,target) in enumerate(dataset):
        dataset[i][1][target_index]=target[target_index]^1 #取反操作
    return dataset

In [11]:
#标签的下标
target_index=31
#身份标签的下标
group_index=20
#shuffle数据集的长度
shuffle_len=200

In [14]:
indices=get_split_indices(celebA_train_dataset,target_index)
celebA_train_target1_dataset=Subset(celebA_train_dataset,indices[0])
celebA_train_target2_dataset=Subset(celebA_train_dataset,indices[1])
#构造一个标签平衡的训练集
length=15000
celebA_train_dataset=ConcatDataset([random_split(celebA_train_target1_dataset,[length,len(celebA_train_target1_dataset)-length])[0],random_split(celebA_train_target2_dataset,[length,len(celebA_train_target2_dataset)-length])[0]])
celebA_train_loader=DataLoader(dataset = celebA_train_dataset,
                              batch_size=batch_size,
                              shuffle=True)
print(len(celebA_train_target1_dataset))
print(len(celebA_train_target2_dataset))

21197
19495


In [15]:
#加载划分数据集
#男女验证集
indices=get_split_indices(celebA_val_dataset,group_index)
celebA_val_male1_dataset=Subset(celebA_val_dataset,indices[0])
celebA_val_male2_dataset=Subset(celebA_val_dataset,indices[1])
celebA_val_male1_loader=DataLoader(dataset = celebA_val_male1_dataset,
                              batch_size=batch_size,
                              shuffle=True)
celebA_val_male2_loader=DataLoader(dataset = celebA_val_male2_dataset,
                              batch_size=batch_size,
                              shuffle=True)
#男女性训练集
indices=get_split_indices(celebA_train_dataset,group_index)
celebA_train_male1_dataset=Subset(celebA_train_dataset,indices[0])
celebA_train_male2_dataset=Subset(celebA_train_dataset,indices[1])

In [16]:
#验证集
#性别1中biglips标签数据集
indices = get_split_indices(celebA_val_male1_dataset, target_index)
celebA_val_male1_biglips1_dataset = Subset(celebA_val_male1_dataset, indices[0])
celebA_val_male1_biglips2_dataset = Subset(celebA_val_male1_dataset, indices[1])
celebA_val_male1_biglips1_loader=DataLoader(dataset = celebA_val_male1_biglips1_dataset,
                              batch_size=batch_size,
                              shuffle=True)
celebA_val_male1_biglips2_loader=DataLoader(dataset = celebA_val_male1_biglips2_dataset,
                              batch_size=batch_size,
                              shuffle=True)
#性别2中biglips标签数据集
indices = get_split_indices(celebA_val_male2_dataset, target_index)
celebA_val_male2_biglips1_dataset = Subset(celebA_val_male2_dataset, indices[0])
celebA_val_male2_biglips2_dataset = Subset(celebA_val_male2_dataset, indices[1])
celebA_val_male2_biglips1_loader=DataLoader(dataset = celebA_val_male2_biglips1_dataset,
                              batch_size=batch_size,
                              shuffle=True)
celebA_val_male2_biglips2_loader=DataLoader(dataset = celebA_val_male2_biglips2_dataset,
                              batch_size=batch_size,
                              shuffle=True)
indices=get_split_indices(celebA_val_dataset,target_index)
celebA_val_biglips1_dataset=Subset(celebA_val_dataset,indices[0])
celebA_val_biglips2_dataset=Subset(celebA_val_dataset,indices[1])
celebA_val_biglips1_loader=DataLoader(dataset =celebA_val_biglips1_dataset,
                              batch_size=batch_size,
                              shuffle=True)
celebA_val_biglips2_loader=DataLoader(dataset =celebA_val_biglips2_dataset,
                              batch_size=batch_size,
                              shuffle=True)

In [17]:
#训练集
#性别1中biglips标签数据集
indices=get_split_indices(celebA_train_male1_dataset,target_index)
celebA_train_male1_biglips1_dataset=Subset(celebA_train_male1_dataset,indices[0])
celebA_train_male1_biglips2_dataset=Subset(celebA_train_male1_dataset,indices[1])
#性别2中biglips标签数据集
indices=get_split_indices(celebA_train_male2_dataset,target_index)
celebA_train_male2_biglips1_dataset=Subset(celebA_train_male2_dataset,indices[0])
celebA_train_male2_biglips2_dataset=Subset(celebA_train_male2_dataset,indices[1])

In [18]:
#shuffle数据集，含50个男正例、反例和女性正例反例的数据集，并打乱biglips
# celebA_train_balance_dataset=ConcatDataset([Subset(celebA_train_male1_biglips1_dataset,range(100)),
#                                             Subset(celebA_train_male1_biglips2_dataset,range(100)),
#                                             Subset(celebA_train_male2_biglips1_dataset,range(100)),
#                                             Subset(celebA_train_male2_biglips2_dataset,range(100))] )
# celebA_train_balance_dataset=ConcatDataset([Subset(celebA_train_male1_dataset,range(100)),
#                                             Subset(celebA_train_male2_dataset,range(100))] )
# celebA_train_balance_dataset=ConcatDataset([random_split(celebA_train_male1_dataset,[100,len(celebA_train_male1_dataset)-100])[0],
#                                             random_split(celebA_train_male2_dataset,[100,len(celebA_train_male2_dataset)-100])[0]] )
celebA_train_balance_dataset=ConcatDataset([random_split(celebA_train_male1_biglips1_dataset,[int(shuffle_len/4),len(celebA_train_male1_biglips1_dataset)-int(shuffle_len/4)])[0],
                                            random_split(celebA_train_male1_biglips2_dataset,[int(shuffle_len/4),len(celebA_train_male1_biglips2_dataset)-int(shuffle_len/4)])[0],
                                            random_split(celebA_train_male2_biglips1_dataset,[int(shuffle_len/4),len(celebA_train_male2_biglips1_dataset)-int(shuffle_len/4)])[0],
                                            random_split(celebA_train_male2_biglips2_dataset,[int(shuffle_len/4),len(celebA_train_male2_biglips2_dataset)-int(shuffle_len/4)])[0]] )
celebA_train_shuffle_biglips_dataset=shuffle_dataset(celebA_train_balance_dataset,target_index=target_index)
celebA_train_balance_dataloader=DataLoader(dataset = celebA_train_balance_dataset,
                              batch_size=batch_size,
                              shuffle=True)
celebA_train_shuffle_biglips_dataloader=DataLoader(dataset = celebA_train_shuffle_biglips_dataset,
                              batch_size=batch_size,
                              shuffle=True)

In [19]:
# 训练模型的方法定义
print('training on: ', device)
def test(loader, net,target_index):
    net.eval()
    acc = 0.0
    sum = 0.0
    loss_sum = 0
    for batch, (data, target) in enumerate(loader):
        data, target = data.to(device), target[:,target_index].to(device)
        output = net(data)
        loss = criterion(output, target)
        loss_sum += loss.item()
        _, predicted = output.max(1)
        sum += target.size(0)
        acc += predicted.eq(target).sum().item()
        # acc += torch.sum(torch.argmax(output, dim=1) == target).item()
        # sum += len(target)
        # loss_sum += loss.item()
    print('test  acc: %.2f%%, loss: %.4f' % (100 * acc / sum, loss_sum / (batch + 1)))
    return 100 * acc / sum, loss_sum / (batch + 1)

def train(loader, model, target_index, training_type):
    '''
    :param loader:
    :param model:
    :param target_index: 标签下标
    :param training_type: 模型名称
    :return:
    '''
    model.train()
    acc = 0.0
    sum = 0.0
    loss_sum = 0

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
    for batch, (data, target) in tqdm.tqdm( enumerate(loader),desc="模型训练中：", total=len(loader)):
        data, target = data.to(device), target[:,target_index].type(torch.LongTensor).to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
        _, predicted = output.max(1)
        sum += target.size(0)
        acc += predicted.eq(target).sum().item()

        # acc += torch.sum(torch.argmax(output, dim=1) == target).item()
        # sum += len(target)
        # loss_sum += loss.item()

        # if batch % 200 == 0:
        #     print('\tbatch: %d, loss: %.4f' % (batch, loss.item()))
    print('train acc: %.2f%%, loss: %.4f' % (100 * acc / sum, loss_sum / (batch + 1)))
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, "../models/" + str(training_type) + "_checkpoint.pth")

def load_model(model_path):
    #加载模型
    net = AlexNet().to(device)
    checkpoint = torch.load(model_path)
    net.load_state_dict(checkpoint['model_state_dict'])
    return net
def CelebA_test(model,target_index):
    print("全部测试集：")
    test(celebA_val_loader,model,target_index=target_index)
    print("性别1测试集：")
    test(celebA_val_male1_loader,model,target_index=target_index)
    print("性别2测试集：")
    test(celebA_val_male2_loader,model,target_index=target_index)
    print("标签1测试集：")
    test(celebA_val_biglips1_loader,model,target_index=target_index)
    print("标签2测试集：")
    test(celebA_val_biglips2_loader,model,target_index=target_index)

training on:  cuda:1


In [ ]:
net=load_model('../models/AlexNet_origin_smiling_checkpoint.pth')

In [ ]:
%%time
#原始训练
for epoch in range(200):
        print('epoch: %d' % epoch)
        train(celebA_train_loader,net,target_index=target_index,training_type="AlexNet_origin_smiling")
        CelebA_test(net,target_index=target_index)

In [21]:
net=load_model('../models/AlexNet_origin_smiling_checkpoint.pth')

In [22]:
CelebA_test(net,target_index=target_index)

全部测试集：
test  acc: 68.87%, loss: 1.0222
性别1测试集：
test  acc: 66.63%, loss: 1.0083
性别2测试集：
test  acc: 72.38%, loss: 1.0479
标签1测试集：
test  acc: 82.53%, loss: 0.9001
标签2测试集：
test  acc: 55.06%, loss: 1.1746


In [ ]:
#混淆训练,celebA数据集target为biglips
for epoch in range(10):
        print('epoch: %d' % epoch)
        train(celebA_train_shuffle_biglips_dataloader,net,target_index=target_index,training_type="celebA_shuffle_bignose")

In [ ]:
net=load_model('models/celebA_shuffle_crop_biglips_checkpoint.pth')

In [13]:
CelebA_test(net,target_index=target_index)

NameError: name 'CelebA_test' is not defined

In [ ]:
#恢复训练,celebA数据集target为biglips
for epoch in range(10):
        print('epoch: %d' % epoch)
        train(celebA_train_balance_dataloader,net,target_index=target_index,training_type="celebA_balance_bignose")

In [ ]:
net=load_model('models/celebA_balance_biglips_checkpoint.pth')

In [ ]:
CelebA_test(net,target_index=target_index)